In [5]:
import yfinance as yf
import pandas as pd
import mysql.connector
from datetime import datetime, timedelta
import logging

In [6]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('trading_system.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [7]:
class YahooDataCollector:
    def __init__(self, db_config):
        self.db_config = db_config
        self.logger = logging.getLogger(__name__) # Use the configured logger

    def connect_db(self):
        try:
            conn = mysql.connector.connect(**self.db_config)
            return conn
        except mysql.connector.Error as e:
            self.logger.error(f"Database connection error: {e}")
            return None

    def create_tables(self):
        """Create necessary database tables if they don't exist."""
        conn = self.connect_db()
        if not conn: return False
        cursor = conn.cursor()

        stock_table = """
        CREATE TABLE IF NOT EXISTS stocks (
            id INT AUTO_INCREMENT PRIMARY KEY,
            symbol VARCHAR(10) UNIQUE NOT NULL,
            company_name VARCHAR(255),
            sector VARCHAR(100),
            is_active BOOLEAN DEFAULT 0,
            added_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """
        price_table = """
        CREATE TABLE IF NOT EXISTS historical_prices (
            id INT AUTO_INCREMENT PRIMARY KEY,
            symbol VARCHAR(10),
            date DATE,
            open_price DECIMAL(10,4), high_price DECIMAL(10,4), low_price DECIMAL(10,4), close_price DECIMAL(10,4),
            volume BIGINT, adj_close DECIMAL(10,4),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            UNIQUE KEY unique_symbol_date (symbol, date),
            FOREIGN KEY (symbol) REFERENCES stocks(symbol) ON DELETE CASCADE ON UPDATE CASCADE
        )
        """
        portfolio_table = """
        CREATE TABLE IF NOT EXISTS portfolio (
            id INT AUTO_INCREMENT PRIMARY KEY,
            symbol VARCHAR(10),
            shares_owned DECIMAL(10,4) DEFAULT 0, avg_cost_basis DECIMAL(10,4) DEFAULT 0,
            total_invested DECIMAL(10,2) DEFAULT 0, current_value DECIMAL(10,2) DEFAULT 0,
            unrealized_pnl DECIMAL(10,2) DEFAULT 0,
            last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
            UNIQUE KEY unique_symbol (symbol),
            FOREIGN KEY (symbol) REFERENCES stocks(symbol) ON DELETE CASCADE ON UPDATE CASCADE
        )
        """
        trades_table = """
        CREATE TABLE IF NOT EXISTS simulated_trades (
            id INT AUTO_INCREMENT PRIMARY KEY,
            symbol VARCHAR(10), action ENUM('BUY', 'SELL'), shares DECIMAL(10,4), price DECIMAL(10,4),
            total_value DECIMAL(10,2), commission DECIMAL(10,2) DEFAULT 0,
            trade_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP, strategy VARCHAR(50), notes TEXT
        )
        """
        account_table = """
        CREATE TABLE IF NOT EXISTS account_balance (
            id INT AUTO_INCREMENT PRIMARY KEY,
            cash_balance DECIMAL(12,2) DEFAULT 100000.00, total_portfolio_value DECIMAL(12,2) DEFAULT 0,
            total_account_value DECIMAL(12,2) DEFAULT 100000.00,
            daily_pnl DECIMAL(10,2) DEFAULT 0, total_pnl DECIMAL(10,2) DEFAULT 0,
            date DATE UNIQUE, created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """
        tables = [stock_table, price_table, portfolio_table, trades_table, account_table]
        try:
            for table in tables: cursor.execute(table)
            conn.commit()
            self.logger.info("Database tables created/verified successfully (with is_active column).")
            return True
        except mysql.connector.Error as e:
            self.logger.error(f"Error creating tables: {e}")
            return False
        finally:
            cursor.close()
            conn.close()

    def add_stock(self, symbol, company_name=None, sector=None, is_active=False):
        """Add or update a stock, ensuring the is_active flag is set."""
        conn = self.connect_db()
        if not conn: return False
        cursor = conn.cursor()

        if not company_name: # Fetch info if needed
            try:
                stock = yf.Ticker(symbol)
                info = stock.info
                company_name = info.get('longName', symbol)
                sector = info.get('sector', 'Unknown')
            except Exception as e:
                self.logger.warning(f"Could not fetch YFinance info for {symbol}: {e}")
                company_name = symbol
                sector = 'Unknown'

        try:
            # Use ON DUPLICATE KEY UPDATE to set flags correctly
            query = """
            INSERT INTO stocks (symbol, company_name, sector, is_active)
            VALUES (%s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                company_name = VALUES(company_name), sector = VALUES(sector), is_active = VALUES(is_active)
            """
            cursor.execute(query, (symbol, company_name, sector, is_active))
            conn.commit()
            status = "activated" if is_active else "added/updated (inactive)"
            self.logger.info(f"Stock {status}: {symbol} - {company_name}")
            return True
        except mysql.connector.Error as e:
            self.logger.error(f"Error adding/updating stock {symbol}: {e}")
            return False
        finally:
            cursor.close()
            conn.close()

    def deactivate_old_stocks(self, active_symbols):
        """Sets is_active=0 for any stock NOT in the active_symbols list."""
        if not active_symbols:
             self.logger.warning("No active symbols provided to deactivate_old_stocks.")
             return False

        conn = self.connect_db()
        if not conn: return False
        cursor = conn.cursor()
        try:
            format_strings = ','.join(['%s'] * len(active_symbols))
            query = f"UPDATE stocks SET is_active = 0 WHERE symbol NOT IN ({format_strings})"
            cursor.execute(query, tuple(active_symbols))
            deactivated_count = cursor.rowcount
            conn.commit()
            self.logger.info(f"Deactivated {deactivated_count} old/unlisted stocks.")
            return True
        except mysql.connector.Error as e:
            self.logger.error(f"Error deactivating old stocks: {e}")
            return False
        finally:
            cursor.close()
            conn.close()

    def fetch_historical_data(self, symbol, period="2y"):
        """Fetch historical data from Yahoo Finance."""
        try:
            stock = yf.Ticker(symbol)
            hist = stock.history(period=period)
            if hist.empty:
                self.logger.warning(f"No data found for {symbol}")
                return None
            hist.reset_index(inplace=True)
            hist['Symbol'] = symbol
            return hist
        except Exception as e:
            self.logger.error(f"Error fetching data for {symbol}: {e}")
            return None

    def store_historical_data(self, symbol, data):
        """Store historical data in database, ignoring duplicates."""
        if data is None or data.empty: return False
        conn = self.connect_db()
        if not conn: return False
        cursor = conn.cursor()
        inserted_count = 0
        try:
            query = """
            INSERT IGNORE INTO historical_prices
            (symbol, date, open_price, high_price, low_price, close_price, volume, adj_close)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """
            values_list = []
            for _, row in data.iterrows():
                 values_list.append((
                    symbol, row['Date'].date(),
                    float(row['Open']), float(row['High']), float(row['Low']), float(row['Close']),
                    int(row['Volume']), float(row['Close']) # Assuming 'Close' is adjusted
                 ))
            cursor.executemany(query, values_list)
            inserted_count = cursor.rowcount # Number of rows actually inserted
            conn.commit()
            self.logger.info(f"Stored {inserted_count} new records for {symbol} (out of {len(data)} fetched).")
            return True
        except mysql.connector.Error as e:
            self.logger.error(f"Error storing data for {symbol}: {e}")
            return False
        finally:
            cursor.close()
            conn.close()

    def update_all_active_stocks(self):
        """Update historical data for all stocks marked as active."""
        conn = self.connect_db()
        if not conn: return False
        cursor = conn.cursor()
        cursor.execute("SELECT symbol FROM stocks WHERE is_active = 1")
        symbols = [row[0] for row in cursor.fetchall()]
        cursor.close()
        conn.close()

        if not symbols:
            self.logger.warning("No active stocks found to update.")
            return False

        self.logger.info(f"Found {len(symbols)} active stocks to update.")
        success_count = 0
        for symbol in symbols:
            self.logger.info(f"Updating data for {symbol}...")
            data = self.fetch_historical_data(symbol)
            if self.store_historical_data(symbol, data):
                success_count += 1
        self.logger.info(f"Successfully updated {success_count}/{len(symbols)} active stocks")
        return success_count == len(symbols)

    def initialize_account(self, starting_balance=100000):
        """Initialize account balance for today and optionally yesterday."""
        conn = self.connect_db()
        if not conn: return False
        cursor = conn.cursor()
        try:
            today_date = datetime.now().date()
            yesterday_date = today_date - timedelta(days=1)

            # Insert/Ignore for today
            query_today = """
            INSERT IGNORE INTO account_balance
            (cash_balance, total_portfolio_value, total_account_value, date)
            VALUES (%s, 0, %s, %s)
            """
            cursor.execute(query_today, (starting_balance, starting_balance, today_date))

            # Check and insert/ignore for yesterday if needed
            cursor.execute("SELECT COUNT(*) FROM account_balance WHERE date = %s", (yesterday_date,))
            if cursor.fetchone()[0] == 0:
                query_yesterday = """
                INSERT IGNORE INTO account_balance
                (cash_balance, total_portfolio_value, total_account_value, date)
                VALUES (%s, 0, %s, %s)
                """
                cursor.execute(query_yesterday, (starting_balance, starting_balance, yesterday_date))
                self.logger.info("Inserted baseline account record for yesterday.")

            conn.commit()
            self.logger.info(f"Account initialized with ${starting_balance:,.2f} for {today_date} (and {yesterday_date} if needed).")
            return True
        except mysql.connector.Error as e:
            self.logger.error(f"Error initializing account: {e}")
            return False
        finally:
            cursor.close()
            conn.close()

In [8]:
if __name__ == "__main__":
    db_config = {
        'host': '127.0.0.1',
        'user': 'root',
        'password': '',
        'database': 'trading_system'
    }

    collector = YahooDataCollector(db_config)

    print("Creating/Verifying database tables...")
    collector.create_tables()

    print("\nInitializing account...")
    collector.initialize_account(100000)

    # Define the 20 target stocks - **REFORMATTED**
    stocks_to_track = [
        ('KR', 'The Kroger Co.', 'Consumer Defensive'),
        ('AAPL', 'Apple Inc.', 'Technology'),
        ('ERO', 'Ero Copper Corp.', 'Basic Materials'),
        ('BLK', 'BlackRock, Inc.', 'Financial Services'),
        ('RSP', 'Invesco S&P 500 Equal Weight ETF', 'ETF'),
        ('JPM', 'JPMorgan Chase & Co.', 'Financial Services'),
        ('FXP', 'ProShares UltraShort FTSE China 50', 'ETF'),
        ('GOOGL', 'Alphabet Inc.', 'Communication Services'),
        ('TSLA', 'Tesla, Inc.', 'Automotive'),
        ('MDT', 'Medtronic plc', 'Healthcare'),
        ('AMZN', 'Amazon.com, Inc.', 'Consumer Cyclical'),
        ('VGK', 'Vanguard FTSE Europe ETF', 'ETF'),
        ('OXY', 'Occidental Petroleum Corporation', 'Energy'),
        ('PGJ', 'Invesco Golden Dragon China ETF', 'ETF'),
        ('GXC', 'SPDR S&P China ETF', 'ETF'),
        ('NVDA', 'NVIDIA Corporation', 'Technology'),
        ('SPY', 'SPDR S&P 500 ETF', 'ETF'),
        ('META', 'Meta Platforms, Inc.', 'Communication Services'),
        ('XPP', 'ProShares Ultra FTSE China 50', 'ETF'),
        ('MSFT', 'Microsoft Corporation', 'Technology'),
    ]
    active_symbols_list = [s[0] for s in stocks_to_track]

    print(f"\nAdding/Activating {len(stocks_to_track)} target stocks...")
    for symbol, name, sector in stocks_to_track:
        collector.add_stock(symbol, name, sector, is_active=True) # Mark as active

    print("\nDeactivating any old stocks not in the target list...")
    collector.deactivate_old_stocks(active_symbols_list) # Clean up old entries

    print("\nFetching/Updating historical data for active stocks... (This may take a minute)")
    collector.update_all_active_stocks()

    print("\n=========================================================")
    print("✓ Data collection setup complete!")
    print(f"✓ Database is now tracking ONLY {len(stocks_to_track)} active stocks.")
    print("✓ Ready to run '02_sentiment_scraper_UPDATED.py'.")
    print("=========================================================")

Creating/Verifying database tables...


2025-10-25 22:39:28,388 - INFO - Database tables created/verified successfully (with is_active column).
2025-10-25 22:39:28,565 - INFO - Inserted baseline account record for yesterday.
2025-10-25 22:39:28,567 - INFO - Account initialized with $100,000.00 for 2025-10-25 (and 2025-10-24 if needed).



Initializing account...

Adding/Activating 20 target stocks...


2025-10-25 22:39:28,767 - INFO - Stock activated: KR - The Kroger Co.
2025-10-25 22:39:28,949 - INFO - Stock activated: AAPL - Apple Inc.
2025-10-25 22:39:29,104 - INFO - Stock activated: ERO - Ero Copper Corp.
2025-10-25 22:39:29,254 - INFO - Stock activated: BLK - BlackRock, Inc.
2025-10-25 22:39:29,469 - INFO - Stock activated: RSP - Invesco S&P 500 Equal Weight ETF
2025-10-25 22:39:29,669 - INFO - Stock activated: JPM - JPMorgan Chase & Co.
2025-10-25 22:39:29,854 - INFO - Stock activated: FXP - ProShares UltraShort FTSE China 50
2025-10-25 22:39:30,029 - INFO - Stock activated: GOOGL - Alphabet Inc.
2025-10-25 22:39:30,159 - INFO - Stock activated: TSLA - Tesla, Inc.
2025-10-25 22:39:30,350 - INFO - Stock activated: MDT - Medtronic plc
2025-10-25 22:39:30,488 - INFO - Stock activated: AMZN - Amazon.com, Inc.
2025-10-25 22:39:30,576 - INFO - Stock activated: VGK - Vanguard FTSE Europe ETF
2025-10-25 22:39:30,674 - INFO - Stock activated: OXY - Occidental Petroleum Corporation
2025-


Deactivating any old stocks not in the target list...

Fetching/Updating historical data for active stocks... (This may take a minute)


2025-10-25 22:39:31,747 - INFO - Found 20 active stocks to update.
2025-10-25 22:39:31,749 - INFO - Updating data for KR...
2025-10-25 22:39:32,514 - INFO - Stored 502 new records for KR (out of 502 fetched).
2025-10-25 22:39:32,529 - INFO - Updating data for AAPL...
2025-10-25 22:39:32,930 - INFO - Stored 502 new records for AAPL (out of 502 fetched).
2025-10-25 22:39:32,932 - INFO - Updating data for ERO...
2025-10-25 22:39:33,217 - INFO - Stored 502 new records for ERO (out of 502 fetched).
2025-10-25 22:39:33,224 - INFO - Updating data for BLK...
2025-10-25 22:39:33,634 - INFO - Stored 502 new records for BLK (out of 502 fetched).
2025-10-25 22:39:33,637 - INFO - Updating data for RSP...
2025-10-25 22:39:33,852 - INFO - Stored 502 new records for RSP (out of 502 fetched).
2025-10-25 22:39:33,852 - INFO - Updating data for JPM...
2025-10-25 22:39:34,242 - INFO - Stored 502 new records for JPM (out of 502 fetched).
2025-10-25 22:39:34,247 - INFO - Updating data for FXP...
2025-10-25 


✓ Data collection setup complete!
✓ Database is now tracking ONLY 20 active stocks.
✓ Ready to run '02_sentiment_scraper_UPDATED.py'.
